In [1]:
# Requirements
! pip install faiss-cpu
! pip install langchain-community
! pip install sentence-transformers

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.2 MB 904.0 kB/s eta 0:00:20
   - -------------------------------------- 0.8/18.2 MB 891.7 kB/s eta 0:00:20
   - -------------------------------------- 0.8/18.2 MB 891.7 kB/s eta 0:00:20
   - -------------------------------------- 0.8/18.2 MB 891.7 kB/s eta 0:00:20
   -- ------------------------------------- 1.0/18.2 MB 695.2 kB/s eta 0:00:25
   -- ------------------------------------- 1.0/18.2 MB 695.2 kB/s eta 0:00:25
   -- ------------------------------------- 1.0/18.2 MB 695.2 kB/s eta 0:00:25
   -- ------------------------------------- 1.0/18.2 MB 695.2 kB/s eta 0:00:25
   --- -----------

In [2]:
#
import os
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # To perform word embedding
from langchain_text_splitters import RecursiveCharacterTextSplitter # This for chunking
from pypdf import PdfReader
import faiss

In [5]:
# Step 1 : Configure the model

# Configure LLM
key = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=key)
llm_model = genai.GenerativeModel('gemini-2.5-flash-lite')

# Configure  Embedding Model
embedding_model =HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5836\1415344547.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model =HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by set

In [9]:
# Step 2 : Loading the PDF file and extract  the text
loaded_file =PdfReader('Rag Chatbot.pdf')

raw_text = ''
for page in loaded_file.pages:
    text_only = page.extract_text()
    if text_only:
        raw_text += text_only

In [10]:
print(raw_text)

Case Study: RAG Chatbot Powered by Google 
Gemini for Smart Document Q&A 
Project Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation 
(RAG) with Gemini 
GitHub Repository: https://github.com/mukul-mschauhan/RAG-Using-Gemini 
Live Demo: https://gemini-rag2025.streamlit.app/ 
 
Problem Statement 
Across industries such as legal, finance, healthcare, and construction, professionals are 
required to extract insights from massive document repositories—contracts, product 
manuals, policies, reports, regulations, and emails. 
Traditional keyword-based search and static FAQs fail to deliver contextual, accurate 
answers. Employees waste hours scanning PDFs and notes, leading to operational 
inefficiencies, poor decision-making, and knowledge silos. 
There’s a critical need for an intelligent assistant that can understand natural language 
questions, reason over domain-specific documents, and deliver precise responses—
instantly. 
 
Business Objective 
To build an en

In [15]:
# Step 3 : Chunking (Create chunks)
splitter = RecursiveCharacterTextSplitter(chunk_size = 300,chunk_overlap = 50)
chunks = splitter.split_text(raw_text)

In [16]:
chunks # given in list form

['Case Study: RAG Chatbot Powered by Google \nGemini for Smart Document Q&A \nProject Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation \n(RAG) with Gemini \nGitHub Repository: https://github.com/mukul-mschauhan/RAG-Using-Gemini',
 'Live Demo: https://gemini-rag2025.streamlit.app/ \n \nProblem Statement \nAcross industries such as legal, finance, healthcare, and construction, professionals are \nrequired to extract insights from massive document repositories—contracts, product \nmanuals, policies, reports, regulations, and emails.',
 'Traditional keyword-based search and static FAQs fail to deliver contextual, accurate \nanswers. Employees waste hours scanning PDFs and notes, leading to operational \ninefficiencies, poor decision-making, and knowledge silos.',
 'There’s a critical need for an intelligent assistant that can understand natural language \nquestions, reason over domain-specific documents, and deliver precise responses—\ninstantly. \n \nBusiness

In [17]:
len(chunks)

16

In [18]:
print(chunks[0])

Case Study: RAG Chatbot Powered by Google 
Gemini for Smart Document Q&A 
Project Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation 
(RAG) with Gemini 
GitHub Repository: https://github.com/mukul-mschauhan/RAG-Using-Gemini


In [19]:
print(chunks[1])

Live Demo: https://gemini-rag2025.streamlit.app/ 
 
Problem Statement 
Across industries such as legal, finance, healthcare, and construction, professionals are 
required to extract insights from massive document repositories—contracts, product 
manuals, policies, reports, regulations, and emails.


In [20]:
# Step 4 : Create FAISS Vector Store
vector_store = FAISS.from_texts(chunks,embedding_model)

In [31]:
# Step 5 : Configure retriever
retriever = vector_store.as_retriever(search_kwargs={"k":5})

In [32]:
# Step 6 :Take the query
query ="Show me the steps to proceed with this project."

In [33]:
# Step 7 : Retrieval (R)
retrived_docs = retriever.get_relevant_documents(query=query)

In [34]:
retrived_docs

[Document(id='3728d9a2-328c-43cd-8e11-e2e91d49d301', metadata={}, page_content='Live Demo: https://gemini-rag2025.streamlit.app/ \n \nProblem Statement \nAcross industries such as legal, finance, healthcare, and construction, professionals are \nrequired to extract insights from massive document repositories—contracts, product \nmanuals, policies, reports, regulations, and emails.'),
 Document(id='4bd7cdaa-8054-4b1b-b2d8-4f285485839b', metadata={}, page_content='4. Ask questions in natural language \n5. Get contextual answers generated by Google Gemini 1.5 Flash using the retrieved \ndocuments \n \nArchitecture Overview \n1. Frontend: Streamlit web UI for uploading files and chat interface'),
 Document(id='3971fbae-1f2f-4764-940c-b09743304077', metadata={}, page_content='Case Study: RAG Chatbot Powered by Google \nGemini for Smart Document Q&A \nProject Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation \n(RAG) with Gemini \nGitHub Repository: https://github

In [35]:
for i in retrived_docs:
    print(i.page_content)

Live Demo: https://gemini-rag2025.streamlit.app/ 
 
Problem Statement 
Across industries such as legal, finance, healthcare, and construction, professionals are 
required to extract insights from massive document repositories—contracts, product 
manuals, policies, reports, regulations, and emails.
4. Ask questions in natural language 
5. Get contextual answers generated by Google Gemini 1.5 Flash using the retrieved 
documents 
 
Architecture Overview 
1. Frontend: Streamlit web UI for uploading files and chat interface
Case Study: RAG Chatbot Powered by Google 
Gemini for Smart Document Q&A 
Project Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation 
(RAG) with Gemini 
GitHub Repository: https://github.com/mukul-mschauhan/RAG-Using-Gemini
5. LLM Integration: Google Gemini 1.5 Flash for natural language generation using 
retrieved chunks as context 
6. Prompt Engineering: Carefully crafted system and user prompts to ensure 
contextual relevance and safety 
 

In [36]:
context = ' '.join([doc.page_content for doc in retrived_docs])
context

'Live Demo: https://gemini-rag2025.streamlit.app/ \n \nProblem Statement \nAcross industries such as legal, finance, healthcare, and construction, professionals are \nrequired to extract insights from massive document repositories—contracts, product \nmanuals, policies, reports, regulations, and emails. 4. Ask questions in natural language \n5. Get contextual answers generated by Google Gemini 1.5 Flash using the retrieved \ndocuments \n \nArchitecture Overview \n1. Frontend: Streamlit web UI for uploading files and chat interface Case Study: RAG Chatbot Powered by Google \nGemini for Smart Document Q&A \nProject Title: Intelligent Document Q&A Assistant using Retrieval-Augmented Generation \n(RAG) with Gemini \nGitHub Repository: https://github.com/mukul-mschauhan/RAG-Using-Gemini 5. LLM Integration: Google Gemini 1.5 Flash for natural language generation using \nretrieved chunks as context \n6. Prompt Engineering: Carefully crafted system and user prompts to ensure \ncontextual relev

In [37]:
# Step 8 : Write a Augmeneted prompt (A)
prompt= f'''You are a helpful assistant using RAG. Here is the context {context}

The query asked by user is as follows = {query}'''

In [38]:
# Step 9: Generation (G)
print(llm_model.generate_content(prompt).text)

Based on the provided information, here are the steps to proceed with this project:

1.  **Set up the Frontend:**
    *   Develop a Streamlit web UI. This will be responsible for allowing users to upload their documents and interact with the chatbot through a chat interface.

2.  **Integrate with a Vector Database:**
    *   Use FAISS as the vector database. This database will store embeddings of the document chunks for efficient retrieval.

3.  **Implement Document Processing (Implicit):**
    *   While not explicitly detailed as a separate step, the system will need to process uploaded documents. This likely involves:
        *   **Text Extraction:** Extracting text from various document formats (e.g., PDFs, Word documents).
        *   **Chunking:** Breaking down large documents into smaller, manageable chunks for embedding.
        *   **Embedding Generation:** Using an embedding model to convert these text chunks into numerical vector representations.
        *   **Storing Embeddi